# Pydantic fundamentals

## Star without pydentic

In [1]:
class Person:
    def __init__(self, name: str,gender: str ,age: int) -> None:
        self.name = name
        self.gender = gender
        self.age = age

person1 = Person(name="Hampus", gender="Male", age=24)
person1


In [4]:
person1.name, person1.age, person1.gender

('Hampus', 24, 'Male')

In [5]:
person2 = Person(name=3621+2819, gender=True, age="Twenty four")
person2


## Validation of our person class

In [ ]:
class Person:
    def __init__(self, name: str,gender: str ,age: int) -> None:
        if not isinstance(name, str):
            raise TypeError(f"Name must be a string not {type(name)}")
        self.name = name
        self.gender = gender
        self.age = age
try:
     Person(name=3.303, gender="Male", age=24)
except TypeError as e:
    print(e)

Name must be a string not <class 'float'>


In [17]:
class Person:
    def __init__(self, name: str,gender: str ,age: int) -> None:
        if not isinstance(name, str):
            raise TypeError(f"Name must be a string not {type(name)}")
        self.name = name
        self.gender = gender
        
        if not isinstance(age,int):
            raise TypeError(f"Age must be an integer not {type(age)}")
        if not 0 <= age < 125:
            raise ValueError(f"Age must be between 0 and 124 not {age}")  
        self.age = age
    
    def __repr__(self):
        return f"Person({self.name}, {self.age}, {self.gender})"
try:
    person3 = Person(name="Bob", gender="Male", age= -65)
except ValueError or TypeError as e:
    print(e)


Age must be between 0 and 124 not -65


In [18]:
person4 = Person(name="Anna", gender="Female", age= 5)
person4

Person(Anna, 5, Female)

In [20]:
person4.age = -5
person4

Person(Anna, -5, Female)

In [23]:
class Person:
    def __init__(self, name: str,gender: str ,age: int) -> None:
        if not isinstance(name, str):
            raise TypeError(f"Name must be a string not {type(name)}")
        self.name = name
        self.gender = gender
        self.age = age
   
    @property
    def age(self):
        return self._age
    
    @age.setter
    def age(self,age):
        if not isinstance(age,int):
            raise TypeError(f"Age must be an integer not {type(age)}")
        if not 0 <= age < 125:
            raise ValueError(f"Age must be between 0 and 124 not {age}")  
        self._age = age
    
    def __repr__(self):
        return f"Person({self.name}, {self.age}, {self.gender})"
    

person5 = Person(name="Wilma", gender="Female", age= 24)
person5

Person(Wilma, 24, Female)

In [26]:
try:
    person5.age = 130
except ValueError as e:
    print(e)

Age must be between 0 and 124 not 130


In [28]:
try:
    Person(name="Wilma", gender="Female", age= "24")
except TypeError as e:
    print(e)

Age must be an integer not <class 'str'>


# Validate using pydantic

In [29]:
from pydantic import BaseModel

class Person(BaseModel):
    name: str
    gender: str
    age: int

person6 = Person(name="Kalle", gender="Male", age=30)
person6

Person(name='Kalle', gender='Male', age=30)

In [31]:
person6.age = 35
person6

Person(name='Kalle', gender='Male', age=35)

In [ ]:
# Problem! 
person6.age = "Thirty"
person6


Person(name='Kalle', gender='Male', age='Thirty')

In [37]:
from pydantic import ValidationError
try:
    Person(name=33232, gender="Male", age=30)
except ValidationError as e:
    print(e)

1 validation error for Person
name
  Input should be a valid string [type=string_type, input_value=33232, input_type=int]
    For further information visit https://errors.pydantic.dev/2.12/v/string_type


In [ ]:
# str "30" is coerced to int 30
Person(name="Jon", gender="Male", age="30")

Person(name='Jon', gender='Male', age=30)

In [56]:
from pydantic import ConfigDict

class Person(BaseModel):
    name: str
    gender: str
    age: int

    model_config = ConfigDict(validate_assignment=True)

person7 = Person(name="Jon", gender="M", age="30")

try:
    person7.age = "Thirty"
except ValidationError as e:
    print(e)

person7.age = 35
person7

1 validation error for Person
age
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='Thirty', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/int_parsing


Person(name='Jon', gender='M', age=35)

In [57]:
try:
    Person(name=3.14, gender=59, age= -10)
except ValidationError as e:
    print(e)

2 validation errors for Person
name
  Input should be a valid string [type=string_type, input_value=3.14, input_type=float]
    For further information visit https://errors.pydantic.dev/2.12/v/string_type
gender
  Input should be a valid string [type=string_type, input_value=59, input_type=int]
    For further information visit https://errors.pydantic.dev/2.12/v/string_type


In [58]:
try:
    Person(name= "Pedro", gender="Male", age= -10)
except ValidationError as e:
    print(e)

## Add age validation

In [59]:
from pydantic import Field
from typing import Literal

class Person(BaseModel):
    name: str
    gender: Literal["M", "F"]
    age: int = Field(gt = -1, le=125)

    model_config = ConfigDict(validate_assignment=True)

try:
    Person(name= "Pedro", gender="Mal", age= -10)
except ValidationError as e:
    print(e)

2 validation errors for Person
gender
  Input should be 'M' or 'F' [type=literal_error, input_value='Mal', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/literal_error
age
  Input should be greater than -1 [type=greater_than, input_value=-10, input_type=int]
    For further information visit https://errors.pydantic.dev/2.12/v/greater_than


## Serialization and Deserialization

In [60]:
# Serialization
person7.model_dump()

{'name': 'Jon', 'gender': 'M', 'age': 35}

In [61]:
import json
with open("person.json", "w") as file:
    json.dump(person7.model_dump(), file)

In [62]:
with open("person.json", "r") as file:
    json_data = file.read()

print(repr(json_data))

# Deserialization
Person.model_validate_json(json_data)

'{"name": "Jon", "gender": "M", "age": 35}'


Person(name='Jon', gender='M', age=35)